In [1]:
import json
import pandas as pd
# from langdetect import detect
import logging

In [2]:
import spacy
from spacy_fastlang import LanguageDetector
import uk_core_news_trf

# nlp = spacy.load("uk_core_news_trf")
nlp = spacy.load('uk_core_news_sm')
if 'language_detector' in nlp.pipe_names:
    nlp.remove_pipe('language_detector')

# Add the new language detector to the pipeline
language_detector = LanguageDetector()
nlp.add_pipe('language_detector')


In [25]:
view  = []
with open('data/books.jsonlines', 'r') as f:
    for line in f:
        data = json.loads(line)
        description_doc = nlp(data['description'])
        if not data['description'].strip() or sum(c.isalpha() for c in data['description']) < 3: 
            logging.debug(f"Skipping book {data['book_id']} because it has no description") 
            continue
        item = {}
        item['book_id'] = data['book_id']
        item['title'] = data['title']
        item['description'] = " ".join(token.text for token in description_doc if not token.is_punct and not token.is_stop and not token.is_space)
        item['language'] = description_doc._.language

        print(item)            
        view.append(item)

df = pd.DataFrame(view)
df.to_parquet('data/books.parquet', index=False)   

In [12]:
# Read the Parquet file
# df = pd.read_parquet('data/books.parquet')
# Print the DataFrame
# print(df)

        book_id                                              title  \
0             0                             Їсти, молитися, кохати   
1             1  Серія книжок про Джуді Муді (комплект із 10 книг)   
2             2                   Три новеллы (комплект из 2 книг)   
3             3  Тренажер мозга. Как развить гибкость мышления ...   
4             4                               У пошуках Івана Сили   
...         ...                                                ...   
241984   241984                               Maps for Lost Lovers   
241985   241985                                    The Round House   
241986   241986                                Diary of a Somebody   
241987   241987                            The Twenty-Seventh City   
241988   241988                                   Children of Ruin   

                                              description  
0       «Їсти, молитися, кохати» - надзвичайно щира ро...  
1       На вас чекає знайомство з кумед

In [3]:
None >3

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [4]:
view  = []
with open('data/reviews.jsonlines', 'r') as f:
    for line in f:
        data = json.loads(line)
        description_doc = nlp(data['review'])
        if not data['review'].strip() or sum(c.isalpha() for c in data['review']) < 3: 
            logging.debug(f"Skipping review {data['review']} because it has no content") 
            continue
        item = {}
        item['book_id'] = data['book_id']
        item['review_id'] = data['book_id']
        item['title'] = data['reviewTitle']
        item['is_positive'] = data.get('rating',0) > 3
        item['description'] = " ".join(token.text for token in description_doc if not token.is_punct and not token.is_stop and not token.is_space)
        item['language'] = description_doc._.language

        print(item)            
        view.append(item)

df = pd.DataFrame(view)
df.to_parquet('data/reviews.parquet', index=False)   

{'book_id': 0, 'review_id': 0, 'title': 'Сердечна книга!', 'is_positive': True, 'description': 'Добралася відомого роману вийшов українською мовою Знаєте отримувала такого пристрасного головою зануреного бірюзово океанічного задоволення читання Книга захопила перших сторінок тримала останньої ниткою серця серце близькою розповідь авторки відображає власну історію життя закоханість якою Елізабет простеляє килим речень крізь подорож континентам островам власної душі книга йшла рук Напрочуд легко написана бурхливо перетікає сердечним рікам робота Щиро справжньому відкриті дитячі долоні книга показує життя жінки віддзеркалює мільйоном промінчиків життя кожної Читаючи книгу слова відгукувалися посмішкою грішне смішне сльозами зіжмакане викинуте назавжди зануренням тунель минулого залишеним смарагдом шовковою стежкою майбутнє смолоскипом кричала щасливе живе Безмежно вдячна Елізабет прожите задоволення читачка Захоплююся сим творінням письменниця Сердечно рекомендую читання Ірина Запотоцька'

In [25]:
import pandas as pd

# Read the Parquet file into a DataFrame
df_reviews = pd.read_parquet('data/reviews.parquet')
positive_reviews = df_reviews[df_reviews['is_positive'] == True].value_counts('book_id').sort_index(ascending=True)
max_reviews = positive_reviews.max()

df_books = pd.read_parquet('data/books.parquet')
df_books = df_books.merge(positive_reviews, on='book_id', how='left')
df_books = df_books.rename(columns={'count': 'positive_reviews_count'})
df_books = df_books.fillna({'positive_reviews_count': 0})
df_books['positive_reviews_score'] = df_books['positive_reviews_count'] / max_reviews

df_books.to_parquet('data/books_enriched.parquet', index=False)   
